In [1]:
import math
import datetime as dt
import numpy as np
import pandas
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns
#import missingno as msno
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import scale, StandardScaler, normalize
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.svm import SVC

In [2]:
source="OnlineRetail.xlsx"
dataset=pd.read_excel(source)
dataset.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
dataset_copy=dataset
dataset.shape

(541909, 8)

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
dataset=dataset.loc[pd.isnull(dataset.CustomerID) == False]

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      406829 non-null object
StockCode      406829 non-null object
Description    406829 non-null object
Quantity       406829 non-null int64
InvoiceDate    406829 non-null datetime64[ns]
UnitPrice      406829 non-null float64
CustomerID     406829 non-null float64
Country        406829 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


In [7]:
dataset.describe(include='all')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
count,406829.0,406829,406829,406829.000000,406829,406829.000000,406829.000000,406829
unique,22190.0,3684,3896,NaN,20460,NaN,NaN,37
top,576339.0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,2011-11-14 15:27:00,NaN,NaN,United Kingdom
freq,542.0,2077,2070,NaN,543,NaN,NaN,361878
first,NaN,NaN,NaN,NaN,2010-12-01 08:26:00,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2011-12-09 12:50:00,NaN,NaN,NaN
mean,NaN,NaN,NaN,12.061303,NaN,3.460471,15287.690570,NaN
std,NaN,NaN,NaN,248.693370,NaN,69.315162,1713.600303,NaN
min,NaN,NaN,NaN,-80995.000000,NaN,0.000000,12346.000000,NaN
25%,NaN,NaN,NaN,2.000000,NaN,1.250000,13953.000000,NaN


In [8]:
#remove canceled orders
dataset=dataset[dataset['Quantity']>0]

In [9]:
#remove rows where customerID are NA
dataset.dropna(subset=['CustomerID'],how='all',inplace=True)

In [10]:
dataset.shape

(397924, 8)

In [11]:
#Summary

In [12]:
#exploring the unique values of each attribute
print("Number of transactions: ", dataset['InvoiceNo'].nunique())
print("Number of products bought: ",dataset['StockCode'].nunique())
print("Number of customers:", dataset['CustomerID'].nunique() )
print("Percentage of customers NA: ", round(dataset['CustomerID'].isnull().sum() * 100 / len(dataset),2),"%" )

Number of transactions:  18536
Number of products bought:  3665
Number of customers: 4339
Percentage of customers NA:  0.0 %


In [13]:
#last date available in our dataset
dataset['InvoiceDate'].max()

Timestamp('2011-12-09 12:50:00')

In [14]:
now = dt.date(2011,12,9)
print(now)

2011-12-09


In [15]:
#create a new column called date which contains the only the date of invoice
dataset['date'] = dataset['InvoiceDate'].dt.date

In [16]:
dataset.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01


#### Recency dataset

In [17]:
#group by customers and check last date of purshace
recency_data = dataset.groupby(by='CustomerID', as_index=False)['date'].max()
recency_data.columns = ['CustomerID','LastPurshaceDate']
recency_data.head()

,CustomerID,LastPurshaceDate
0,12346.0,2011-01-18
1,12347.0,2011-12-07
2,12348.0,2011-09-25
3,12349.0,2011-11-21
4,12350.0,2011-02-02


In [18]:
#calculate recency
recency_data['Recency'] = recency_data['LastPurshaceDate'].apply(lambda x: (now - x).days)

In [19]:
recency_data.head()

,CustomerID,LastPurshaceDate,Recency
0,12346.0,2011-01-18,325
1,12347.0,2011-12-07,2
2,12348.0,2011-09-25,75
3,12349.0,2011-11-21,18
4,12350.0,2011-02-02,310


In [20]:
#drop LastPurchaseDate as we don't need it anymore
recency_data.drop('LastPurshaceDate',axis=1,inplace=True)

In [21]:
recency_data.head()

,CustomerID,Recency
0,12346.0,325
1,12347.0,2
2,12348.0,75
3,12349.0,18
4,12350.0,310


In [22]:
data_copy=dataset
# drop duplicates
data_copy.drop_duplicates(subset=['InvoiceNo', 'CustomerID'], keep="first", inplace=True)
#calculate frequency of purchases
frequency_data = data_copy.groupby(by=['CustomerID'], as_index=False)['InvoiceNo'].count()
frequency_data.columns = ['CustomerID','Frequency']
frequency_data.head()

,CustomerID,Frequency
0,12346.0,1
1,12347.0,7
2,12348.0,4
3,12349.0,1
4,12350.0,1


In [23]:
#create column total cost
dataset['TotalCost'] = dataset['Quantity'] * dataset['UnitPrice']

In [24]:
monetary_data = dataset.groupby(by='CustomerID',as_index=False).agg({'TotalCost': 'sum'})
monetary_data.columns = ['CustomerID','Monetary']
monetary_data.head()

,CustomerID,Monetary
0,12346.0,77183.60
1,12347.0,163.16
2,12348.0,331.36
3,12349.0,15.00
4,12350.0,25.20


In [25]:
rfm_data = recency_data.merge(frequency_data.merge(monetary_data,on='CustomerID'),on='CustomerID')
rfm_data.head()
rfm_data.shape

(4339, 4)

In [26]:
dataset_new=rfm_data.merge(dataset,on='CustomerID')
dataset_new.head()

,CustomerID,Recency,Frequency,Monetary,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country,date,TotalCost
0,12346.0,325,1,77183.60,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,United Kingdom,2011-01-18,77183.60
1,12347.0,2,7,163.16,537626,85116,BLACK CANDELABRA T-LIGHT HOLDER,12,2010-12-07 14:57:00,2.10,Iceland,2010-12-07,25.20
2,12347.0,2,7,163.16,542237,84625A,PINK NEW BAROQUECANDLESTICK CANDLE,24,2011-01-26 14:30:00,0.85,Iceland,2011-01-26,20.40
3,12347.0,2,7,163.16,549222,22376,AIRLINE BAG VINTAGE JET SET WHITE,4,2011-04-07 10:43:00,4.25,Iceland,2011-04-07,17.00
4,12347.0,2,7,163.16,556201,23084,RABBIT NIGHT LIGHT,12,2011-06-09 13:01:00,2.08,Iceland,2011-06-09,24.96


In [27]:
dataset_new.shape

(18536, 13)

In [28]:

from sklearn import model_selection, preprocessing, metrics
import lightgbm as lgb

In [29]:
dataset_new=dataset_new.drop(['StockCode','Description','Country','InvoiceDate','date'],axis=1)
dataset_new=dataset_new.drop(['InvoiceNo'],axis=1)

In [30]:
dataset_new.head()

,CustomerID,Recency,Frequency,Monetary,Quantity,UnitPrice,TotalCost
0,12346.0,325,1,77183.60,74215,1.04,77183.60
1,12347.0,2,7,163.16,12,2.10,25.20
2,12347.0,2,7,163.16,24,0.85,20.40
3,12347.0,2,7,163.16,4,4.25,17.00
4,12347.0,2,7,163.16,12,2.08,24.96


In [31]:
from sklearn.model_selection import train_test_split as tts
X=dataset_new.drop(['TotalCost'],axis=1)
Y=dataset_new['TotalCost']
X_train, X_test, y_train, y_test = tts(
    X,
    Y,
    test_size=0.3,
    random_state=0)
y_train.shape
test_id=X_test['CustomerID']

In [32]:
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
            "objective" : "regression",
            "metric" : "rmse", 
            "num_leaves" : 30,
            "min_child_samples" : 100,
            "learning_rate" : 0.1,
            "bagging_fraction" : 0.7,
            "feature_fraction" : 0.5,
            "bagging_freq" : 5,
            "bagging_seed" : 2018,
            "verbosity" : -1

            }

    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    model = lgb.train(params, lgtrain, 1000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=100)

    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_val_y = model.predict(val_X, num_iteration=model.best_iteration)
    return pred_test_y, model, pred_val_y

# Training the model #
pred_test, model, pred_val = run_lgb(X_train, y_train, X_test,y_test, X_test)

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 263.538
Early stopping, best iteration is:
[8]	valid_0's rmse: 134.821


In [33]:
from sklearn import metrics
sub_df = pd.DataFrame({"CustomerID":test_id})
pred_test[pred_test<0] = 0
sub_df["TotalCost"] = np.expm1(pred_test)
sub_df = sub_df.groupby("CustomerID")["TotalCost"].sum().reset_index()
sub_df.columns = ["CustomerID", "TotalCost"]
sub_df["TotalCost"] = np.log1p(sub_df["TotalCost"])
sub_df.to_csv("full_baseline_lgb.csv", index=False)


C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in expm1
  after removing the cwd from sys.path.


In [34]:
sub_df.head()

,CustomerID,TotalCost
0,12348.0,70.673541
1,12349.0,44.087862
2,12352.0,175.171249
3,12356.0,51.869152
4,12359.0,44.913671
